In [1]:
import jsonlines
import torchaudio
from pathlib import Path
import torch, random
import librosa, os
import IPython.display as ipd
from dotenv import load_dotenv
from spellchecker import SpellChecker
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
from collections import defaultdict
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

In [2]:
import jsonlines
import pandas as pd

data = {'image': [], 'caption': [], 'bbox': []}
load_dotenv()

TEAM_NAME = os.getenv("TEAM_NAME", "7up")
TEAM_TRACK = os.getenv("TEAM_TRACK", "advanced")


input_dir = Path(f"/home/jupyter/{TEAM_TRACK}")
with jsonlines.open(input_dir / "vlm.jsonl") as reader:
    for obj in reader:
        image = obj['image']
        annotations = obj['annotations']
        for annotation in annotations:
            caption = annotation['caption']
            bbox = annotation['bbox']
            data['image'].append(image)
            data['caption'].append(caption)
            data['bbox'].append(bbox)

df = pd.DataFrame(data)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.head()

,image,caption,bbox
11011,image_2018.jpg,orange and black fighter jet,"[244, 40, 56, 60]"
26255,image_4808.jpg,green and brown camouflage fighter jet,"[1016, 300, 80, 52]"
19592,image_3594.jpg,white light aircraft,"[676, 336, 76, 44]"
9683,image_1779.jpg,black fighter plane,"[920, 488, 56, 40]"
23,image_4.jpg,"white, red, and green fighter plane","[260, 232, 60, 56]"


In [3]:
from transformers import AutoTokenizer, AutoModelForZeroShotObjectDetection, TrainingArguments, Trainer
model_name = "IDEA-Research/grounding-dino-tiny"
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, transforms=None, max_caption_length=64):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.transforms = transforms
        self.max_caption_length = max_caption_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx]['image']
        caption = self.dataframe.iloc[idx]['caption']
        bbox = self.dataframe.iloc[idx]['bbox']

        image = Image.open("/home/jupyter/advanced/images/" + image_path).convert("RGB")
        if self.transforms:
            image = self.transforms(image)
            # crop the image based on bbox
        # print(f"Image shape: {image.shape}")
     
        encoding = self.tokenizer(caption, return_tensors="pt", padding='max_length', truncation=True, max_length=self.max_caption_length)
        encoding['pixel_values'] = image
        # encoding['bbox'] = torch.tensor(bbox)
        # print(f"BBox shape before padding: {encoding['bbox']}")
        # print(encoding['input_ids'].shape)
        # print(encoding['pixel_values'].shape)
        # print(encoding['attention_mask'].shape)
        return encoding

from torchvision import transforms

data_transforms = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# 3600 -> 50626
# 256 -> 257

In [5]:
train_dataset = CustomDataset(train_df, tokenizer, transforms=data_transforms)
val_dataset = CustomDataset(val_df, tokenizer, transforms=data_transforms)
# Iterate over the first three samples in the train_dataset
for i in range(3):
    sample = train_dataset[i]
    x, y =sample['input_ids'].shape
    print(x)
    print(f"Sample {i + 1}:")
    print(f"Pixel Values Shape: {sample['pixel_values'].shape}")
    print(f"Input IDs Shape: {sample['input_ids'].shape}")
    print(f"Attention Mask Shape: {sample['attention_mask'].shape}")
    print()


1
Sample 1:
Pixel Values Shape: torch.Size([3, 870, 1520])
Input IDs Shape: torch.Size([1, 64])
Attention Mask Shape: torch.Size([1, 64])

1
Sample 2:
Pixel Values Shape: torch.Size([3, 870, 1520])
Input IDs Shape: torch.Size([1, 64])
Attention Mask Shape: torch.Size([1, 64])

1
Sample 3:
Pixel Values Shape: torch.Size([3, 870, 1520])
Input IDs Shape: torch.Size([1, 64])
Attention Mask Shape: torch.Size([1, 64])



In [15]:
from transformers import DefaultDataCollator
training_args = TrainingArguments(
    output_dir="./test_owlvit",
    eval_strategy="epoch",
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',          
    logging_steps=10,
)

data_collator = DefaultDataCollator()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

trainer.train()

ValueError: too many values to unpack (expected 2)

In [17]:
torch.cuda.empty_cache()